## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
drive.mount('/content/drive/')
data = pd.read_csv(r"/content/drive/MyDrive/ML/sms_spam.csv")
data

Mounted at /content/drive/


,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [ ]:
stop_words = stopwords.words('english')
lem = WordNetLemmatizer()


def process(text):
  text = nltk.word_tokenize(text)
  text = [word.lower() for word in text if word.isalpha()]
  text = [word for word in text if word not in stop_words]
  text = [lem.lemmatize(word) for word in text]
  return ' '.join(text)


data['text'] = data['text'].apply(process)
data

,type,text
0,ham,hope good week checking
1,ham,back thanks
2,ham,also cbe pay
3,spam,complimentary star ibiza holiday cash need urg...
4,spam,okmail dear dave final notice collect tenerife...
...,...,...
5554,ham,great role model giving much really wish day m...
5555,ham,awesome remember last time got somebody high f...
5556,spam,prize go another customer c polo ltd suite lon...
5557,spam,sm ac jsco energy high u may know ur leadershi...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['type'], train_size = 0.7)

In [ ]:
#Мешок n-грамм
Nreport = 0
accuracy = 0
bestBag = None
model = MultinomialNB()
for n in [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5),
          (2, 2), (2, 3), (2, 4), (2, 5), (3, 3),
          (3, 4), (3, 5), (4, 4), (4, 5), (5, 5)]:
  vectorizer = CountVectorizer(ngram_range = n)
  X_train_vector = vectorizer.fit_transform(X_train)
  X_test_vector = vectorizer.transform(X_test)
  model.fit(X_train_vector, y_train)
  pred = model.predict(X_test_vector)
  class_report = classification_report(y_test, pred, output_dict=True)
  if class_report['accuracy'] > accuracy:
    accuracy = class_report['accuracy']
    Nreport = class_report
    bestBag = n

In [ ]:
#tf-idf тест
TESTbestAccuracy = 0
bestMin = 0
bestMax = 0
min_treshhold = np.linspace(0, 1, 21)
max_treshhold = np.linspace(0, 1, 21)
model = MultinomialNB()
for min in min_treshhold:
  for max in max_treshhold:
    try:
      vectorizer = TfidfVectorizer(min_df = min, max_df = max)
      X_train_vector = vectorizer.fit_transform(X_train)
      X_test_vector = vectorizer.transform(X_test)
      model.fit(X_train_vector, y_train)
      pred = model.predict(X_test_vector)
      print(accuracy_score(y_test, pred), min, max)
      if accuracy_score(y_test, pred) > TESTbestAccuracy:
        TESTbestAccuracy = accuracy_score(y_test, pred)
        bestMin = min
        bestMax = max
    except ValueError:
      continue
print(TESTbestAccuracy, bestMin, bestMax)
###
### max_df в TfidfVectorizer(min_df = min, max_df = max) ни на что не влияет, так что ниже убираю цикл для max
### да и вообще почему то лучший результат без min_df. при min_df >= 0.1 выдаёт ошибку
###

0.9670263788968825 0.0 0.05
0.9670263788968825 0.0 0.1
0.9670263788968825 0.0 0.15000000000000002
0.9670263788968825 0.0 0.2
0.9670263788968825 0.0 0.25
0.9670263788968825 0.0 0.30000000000000004
0.9670263788968825 0.0 0.35000000000000003
0.9670263788968825 0.0 0.4
0.9670263788968825 0.0 0.45
0.9670263788968825 0.0 0.5
0.9670263788968825 0.0 0.55
0.9670263788968825 0.0 0.6000000000000001
0.9670263788968825 0.0 0.65
0.9670263788968825 0.0 0.7000000000000001
0.9670263788968825 0.0 0.75
0.9670263788968825 0.0 0.8
0.9670263788968825 0.0 0.8500000000000001
0.9670263788968825 0.0 0.9
0.9670263788968825 0.0 0.9500000000000001
0.9670263788968825 0.0 1.0
0.8693045563549161 0.05 0.1
0.8693045563549161 0.05 0.15000000000000002
0.8693045563549161 0.05 0.2
0.8693045563549161 0.05 0.25
0.8693045563549161 0.05 0.30000000000000004
0.8693045563549161 0.05 0.35000000000000003
0.8693045563549161 0.05 0.4
0.8693045563549161 0.05 0.45
0.8693045563549161 0.05 0.5
0.8693045563549161 0.05 0.55
0.8693045563549

In [ ]:
#tf-idf
TFreport = 0
accuracy = 0
bestTF = 0
min_treshhold = np.linspace(0, 1, 101)
model = MultinomialNB()
for min in min_treshhold:
  try:
    vectorizer = TfidfVectorizer(min_df = min)
    X_train_vector = vectorizer.fit_transform(X_train)
    X_test_vector = vectorizer.transform(X_test)
    model.fit(X_train_vector, y_train)
    pred = model.predict(X_test_vector)
    class_report = classification_report(y_test, pred, output_dict=True)
    if class_report['accuracy'] > accuracy:
      accuracy = class_report['accuracy']
      TFreport = class_report
      bestTF = min
  except ValueError:
    continue
print(bestTF)

0.0


In [ ]:
#nchar
Creport = 0
accuracy = 0
nсhar = None
model = MultinomialNB()
idx = []
for i in range(1, 11):
  for j in range(i, 11):
    idx.append((i,j))
for n in idx:
  vectorizer = CountVectorizer(analyzer = 'char', ngram_range = n)
  X_train_vector = vectorizer.fit_transform(X_train)
  X_test_vector = vectorizer.transform(X_test)
  model.fit(X_train_vector, y_train)
  pred = model.predict(X_test_vector)
  class_report = classification_report(y_test, pred, output_dict=True)
  if class_report['accuracy'] > accuracy:
      accuracy = class_report['accuracy']
      Creport = class_report
      nchar = n

In [ ]:
print(bestBag, bestTF, nchar)
df = pd.DataFrame(columns = ['precision', 'recall', 'f1-score', 'support'])
df = df.append(Nreport['weighted avg'], ignore_index=True)
df = df.append(TFreport['weighted avg'], ignore_index=True)
df = df.append(Creport['weighted avg'], ignore_index=True)
df['accuracy'] = [Nreport['accuracy'], TFreport['accuracy'], Creport['accuracy']]
df = df.drop('support', axis = 1).rename({0:'Мешок', 1:'tf-idf', 2:'Char'})
df

(1, 3) 0.0 (1, 10)


,precision,recall,f1-score,accuracy
Мешок,0.980621,0.980815,0.980461,0.980815
tf-idf,0.968231,0.967026,0.964953,0.967026
Char,0.972541,0.973022,0.972610,0.973022


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
text = 'looooong word a lot of words here yeah sure'
re.findall(r'\b\w\w', text)

['lo', 'wo', 'lo', 'of', 'wo', 'he', 'ye', 'su']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = 'text text text. text text. text. TEXT.'
re.split('\.', text, maxsplit=2)

['text text text', ' text text', ' text. TEXT.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = '7 утра а я делаю регулярки УРА 0 часов сна. А можно 100 баллов?'
re.sub('\d', 'DIG', text)

'DIG утра а я делаю регулярки УРА DIG часов сна. А можно DIGDIGDIG баллов?'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
text = 'ссылка? https://colab.research.google.com/drive/1w0egdRKGoFNRDlQWSsKGqEbkP9YcHJNF?usp=sharing'
re.sub('http.+', '', text)

'ссылка? '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
exp = re.compile(r'\b\w{4,}')
exp.findall('again many many words with a lot of letters')

['again', 'many', 'many', 'words', 'with', 'letters']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
exp = re.compile('(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z0-9][a-z0-9-]{0,61}[a-z0-9]')
exp.findall('abc.test@gmail.com xyz@test.in test.first@analyticsvidhya.com first.test@rest.biz')

['abc.test',
 'gmail.com',
 'test.in',
 'test.first',
 'analyticsvidhya.com',
 'first.test',
 'rest.biz']